# Chromatic Orchestrator Results Analysis

This notebook provides interactive analysis and visualization of the Chromatic Orchestrator pipeline results.

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import random
from typing import List, Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("📦 Libraries loaded successfully!")

In [ ]:
class ChromaticResultsAnalyzer:
    """Analyzer for Chromatic Orchestrator pipeline results."""
    
    def __init__(self, results_path: str):
        """Initialize with path to pipeline_results.json"""
        self.results_path = Path(results_path)
        self.raw_results = None
        self.df = None
        self.load_results()
    
    def load_results(self):
        """Load and parse the pipeline results."""
        if not self.results_path.exists():
            raise FileNotFoundError(f"Results file not found: {self.results_path}")
        
        with open(self.results_path, 'r') as f:
            self.raw_results = json.load(f)
        
        # Convert to DataFrame for easier analysis
        self._create_dataframe()
        
        print(f"✅ Loaded {len(self.raw_results)} results from {self.results_path}")
        self.print_summary()
    
    def _create_dataframe(self):
        """Create a pandas DataFrame from the results."""
        rows = []
        
        for result in self.raw_results:
            row = {
                'image_path': result['image_path'],
                'filename': Path(result['image_path']).name,
            }
            
            # Classification data
            if 'classification' in result and result['classification']:
                row.update({
                    'category': result['classification']['category'],
                    'classification_tokens': result['classification']['input_tokens'] + result['classification']['output_tokens']
                })
            
            # Scoring data
            if 'scoring' in result and result['scoring']:
                row.update({
                    'score': result['scoring']['score'],
                    'reasoning': result['scoring'].get('reasoning', ''),
                    'scoring_tokens': result['scoring']['input_tokens'] + result['scoring']['output_tokens']
                })
            
            # Summary data
            if 'summary' in result and result['summary']:
                row.update({
                    'summary': result['summary']['summary'],
                    'summary_length': len(result['summary']['summary']),
                    'summary_tokens': result['summary']['input_tokens'] + result['summary']['output_tokens']
                })
            
            rows.append(row)
        
        self.df = pd.DataFrame(rows)
        
        # Fill NaN values for missing stages
        if 'score' in self.df.columns:
            self.df['score'] = self.df['score'].fillna(-1)
        if 'category' in self.df.columns:
            self.df['category'] = self.df['category'].fillna('UNKNOWN')
    
    def print_summary(self):
        """Print a summary of the loaded results."""
        print("\n📊 Results Summary:")
        print(f"  Total images: {len(self.df)}")
        
        if 'category' in self.df.columns:
            print(f"  Categories: {self.df['category'].value_counts().to_dict()}")
        
        if 'score' in self.df.columns:
            valid_scores = self.df[self.df['score'] >= -2]['score']
            if len(valid_scores) > 0:
                print(f"  Score range: {valid_scores.min():.1f} - {valid_scores.max():.1f}")
                print(f"  Average score: {valid_scores.mean():.2f}")
        
        if 'summary' in self.df.columns:
            summarized = self.df['summary'].notna().sum()
            print(f"  Summarized: {summarized} images")
    
    def get_sorted_by_score(self, ascending=False, category=None, min_score=None, max_score=None):
        """Get results sorted by score with optional filtering."""
        df_filtered = self.df.copy()
        
        # Filter by category
        if category:
            df_filtered = df_filtered[df_filtered['category'] == category]
        
        # Filter by score range
        if 'score' in df_filtered.columns:
            if min_score is not None:
                df_filtered = df_filtered[df_filtered['score'] >= min_score]
            if max_score is not None:
                df_filtered = df_filtered[df_filtered['score'] <= max_score]
            
            # Sort by score
            df_filtered = df_filtered.sort_values('score', ascending=ascending)
        
        return df_filtered
    
    def sample_random(self, n=9, category=None, min_score=None, max_score=None):
        """Get random sample with optional filtering."""
        df_filtered = self.df.copy()
        
        # Apply filters
        if category:
            df_filtered = df_filtered[df_filtered['category'] == category]
        
        if 'score' in df_filtered.columns:
            if min_score is not None:
                df_filtered = df_filtered[df_filtered['score'] >= min_score]
            if max_score is not None:
                df_filtered = df_filtered[df_filtered['score'] <= max_score]
        
        # Sample randomly
        if len(df_filtered) < n:
            print(f"⚠️  Only {len(df_filtered)} images available (requested {n})")
            return df_filtered
        
        return df_filtered.sample(n=n)


# CHANGE THIS PATH TO YOUR RESULTS FILE
RESULTS_PATH = "../run/rank_0/pipeline_results.json"

try:
    analyzer = ChromaticResultsAnalyzer(RESULTS_PATH)
except FileNotFoundError as e:
    print(f"❌ {e}")
    print("\n🔧 Please update RESULTS_PATH to point to your pipeline_results.json file")
    analyzer = None

In [ ]:
analyzer

In [ ]:
# 🎯 CHANGE THESE VARIABLES TO EXPLORE YOUR RESULTS

N_IMAGES = 27         # Number of images to display
GRID_COLS = 9        # Grid layout (columns)
SORT_BY = 'random'   # 'random', 'score_high', 'score_low'
CATEGORY = None      # Filter by category (None, 'A', 'B', etc.)
MIN_SCORE = None     # Score filters (None to disable)
MAX_SCORE = None

In [ ]:
df_sorted = analyzer.get_sorted_by_score(ascending=True, category=CATEGORY, min_score=MIN_SCORE, max_score=MAX_SCORE)

In [ ]:
df_sorted

In [ ]:
df_sorted['reasoning']

In [ ]:
# 📈 Score distribution analysis
if analyzer and 'score' in analyzer.df.columns:
    valid_scores = analyzer.df[analyzer.df['score'] >= -1]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Overall score histogram
    ax1.hist(valid_scores['score'], bins=20, alpha=0.7, edgecolor='black')
    ax1.axvline(valid_scores['score'].mean(), color='red', linestyle='--', 
                label=f'Mean: {valid_scores["score"].mean():.2f}')
    ax1.set_title('Score Distribution')
    ax1.set_xlabel('Score')
    ax1.set_ylabel('Count')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Box plot by category
    if 'category' in valid_scores.columns:
        sns.boxplot(data=valid_scores, x='category', y='score', ax=ax2)
        ax2.set_title('Score Distribution by Category')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print some stats
    print(f"\n📊 Score Statistics:")
    print(f"  Total scored images: {len(valid_scores)}")
    print(f"  Average score: {valid_scores['score'].mean():.2f}")
    print(f"  Score range: {valid_scores['score'].min():.1f} - {valid_scores['score'].max():.1f}")
    
    # Count by score ranges
    excellent = len(valid_scores[valid_scores['score'] >= 8])
    good = len(valid_scores[(valid_scores['score'] >= 6) & (valid_scores['score'] < 8)])
    fair = len(valid_scores[(valid_scores['score'] >= 4) & (valid_scores['score'] < 6)])
    poor = len(valid_scores[valid_scores['score'] < 4])
    
    print(f"\n🏆 Quality Breakdown:")
    print(f"  Excellent (8-10): {excellent} images")
    print(f"  Good (6-8): {good} images")
    print(f"  Fair (4-6): {fair} images")
    print(f"  Poor (0-4): {poor} images")

In [ ]:
def display_image_grid_robust(df_subset, 
                             grid_cols=3, 
                             figsize_per_image=(4, 4), 
                             show_score=True,
                             show_category=True,
                             show_filename=False,
                             title="Image Grid"):
    """Display a grid of images with robust error handling."""
    if len(df_subset) == 0:
        print("❌ No images to display")
        return
    
    try:
        n_images = len(df_subset)
        
        # Handle single image case
        if n_images == 1:
            grid_cols = 1
            grid_rows = 1
        else:
            grid_rows = (n_images + grid_cols - 1) // grid_cols
        
        fig_width = grid_cols * figsize_per_image[0]
        fig_height = grid_rows * figsize_per_image[1]
        
        fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(fig_width, fig_height))
        fig.suptitle(title, fontsize=16, fontweight='bold')
        
        # Handle different subplot configurations
        if n_images == 1:
            axes = [axes]
        elif grid_rows == 1 and grid_cols > 1:
            axes = axes.flatten()
        elif grid_cols == 1 and grid_rows > 1:
            axes = axes.flatten()
        elif grid_rows > 1 and grid_cols > 1:
            axes = axes.flatten()
        else:
            axes = [axes]
        
        successful_displays = 0
        
        for idx, (_, row) in enumerate(df_subset.iterrows()):
            if idx >= len(axes):
                break
                
            ax = axes[idx]
            
            # Try to load and display image
            try:
                img_path = '../' + row['image_path']
                if not Path(img_path).exists():
                    raise FileNotFoundError(f"Image not found: {img_path}")
                
                img = Image.open(img_path)
                ax.imshow(img)
                ax.axis('off')
                
                # Build title components
                title_parts = []
                
                if show_score and 'score' in row and pd.notna(row['score']) and row['score'] >= -2:
                    title_parts.append(f"Score: {row['score']:.1f}")
                
                if show_category and 'category' in row and pd.notna(row['category']):
                    title_parts.append(f"Cat: {row['category']}")
                
                if show_filename and 'filename' in row:
                    filename = str(row['filename'])
                    filename = filename[:15] + "..." if len(filename) > 18 else filename
                    title_parts.append(filename)
                
                if title_parts:
                    ax.set_title('\n'.join(title_parts), fontsize=10, pad=10)
                
                successful_displays += 1
                
            except Exception as e:
                # Handle individual image errors gracefully
                ax.text(0.5, 0.5, f"Error loading\n{row.get('filename', 'Unknown')}\n{str(e)[:30]}...", 
                       ha='center', va='center', transform=ax.transAxes, fontsize=8)
                ax.axis('off')
                print(f"⚠️  Error loading {row.get('filename', 'Unknown')}: {e}")
        
        # Hide empty subplots
        for idx in range(n_images, len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        if successful_displays < n_images:
            print(f"⚠️  Successfully displayed {successful_displays}/{n_images} images")
        
    except Exception as e:
        print(f"❌ Error creating image grid: {e}")
        print("📋 Fallback: Listing available images instead:")
        for idx, (_, row) in enumerate(df_subset.iterrows()):
            score_str = f"Score: {row.get('score', 'N/A')}" if 'score' in row else ""
            cat_str = f"Category: {row.get('category', 'N/A')}" if 'category' in row else ""
            print(f"  {idx+1}. {row.get('filename', 'Unknown')} | {score_str} | {cat_str}")

In [ ]:
def smart_grid_robust(category=None, n_images=9, grid_cols=3, sort_by='random', 
                     min_score=None, max_score=None):
    """Smart grid with enhanced error handling."""
    if analyzer is None:
        print("❌ No results loaded")
        return
    
    try:
        # Check how many images are available with current filters
        if category:
            available = analyzer.df[analyzer.df['category'] == category].copy()
        else:
            available = analyzer.df.copy()
        
        # Apply score filters
        if 'score' in available.columns:
            if min_score is not None:
                available = available[available['score'] >= min_score]
            if max_score is not None:
                available = available[available['score'] <= max_score]
        
        available_count = len(available)
        
        if available_count == 0:
            print(f"❌ No images found with the specified filters")
            if category:
                print(f"   Category: {category}")
            if min_score is not None:
                print(f"   Min score: {min_score}")
            if max_score is not None:
                print(f"   Max score: {max_score}")
            return
        
        # Auto-adjust grid size if needed
        actual_n = min(n_images, available_count)
        
        if actual_n < n_images:
            print(f"📊 Showing all {actual_n} available images (requested {n_images})")
        
        # Auto-adjust grid columns for better layout
        if actual_n == 1:
            grid_cols = 1
        elif actual_n <= 4:
            grid_cols = min(2, actual_n)
        elif actual_n <= 9:
            grid_cols = 3
        elif actual_n <= 16:
            grid_cols = 4
        
        # Get the data based on sort preference
        if sort_by == 'random':
            df_subset = available.sample(n=actual_n) if len(available) >= actual_n else available
            title = f"Random Sample ({len(df_subset)} images)"
        elif sort_by == 'score_high':
            df_subset = available.sort_values('score', ascending=False).head(actual_n)
            title = f"Highest Scores ({len(df_subset)} images)"
        elif sort_by == 'score_low':
            df_subset = available.sort_values('score', ascending=True).head(actual_n)
            title = f"Lowest Scores ({len(df_subset)} images)"
        else:
            print(f"❌ Unknown sort_by: {sort_by}. Using random.")
            df_subset = available.sample(n=actual_n) if len(available) >= actual_n else available
            title = f"Random Sample ({len(df_subset)} images)"
        
        # Add filter info to title
        filter_info = []
        if category:
            filter_info.append(f"Category {category}")
        if min_score is not None:
            filter_info.append(f"Score ≥ {min_score}")
        if max_score is not None:
            filter_info.append(f"Score ≤ {max_score}")
        
        if filter_info:
            title += f" | {', '.join(filter_info)}"
        
        display_image_grid_robust(df_subset, grid_cols=grid_cols, title=title)
        
    except Exception as e:
        print(f"❌ Error in smart_grid_robust: {e}")
        print(f"   Attempted filters - Category: {category}, Min score: {min_score}, Max score: {max_score}")

In [ ]:
def explore_all_categories_robust(n_images_per_category=16, 
                                 sort_by='score_high',
                                 min_score=None,
                                 max_score=None,
                                 skip_unknown=True):
    """Loop through all categories with comprehensive error handling."""
    if analyzer is None:
        print("❌ No results loaded")
        return
    
    if 'category' not in analyzer.df.columns:
        print("❌ No category data available")
        return
    
    try:
        # Get all unique categories
        categories = sorted(analyzer.df['category'].unique())
        
        if skip_unknown and 'UNKNOWN' in categories:
            categories.remove('UNKNOWN')
        
        print(f"🔍 Exploring {len(categories)} categories...")
        print("=" * 60)
        
        successful_categories = 0
        
        for i, category in enumerate(categories):
            try:
                print(f"\n🏷️  Category {category}")
                print("-" * 40)
                
                # Count available images for this category
                cat_data = analyzer.df[analyzer.df['category'] == category].copy()
                
                # Apply score filters to get actual count
                if 'score' in cat_data.columns:
                    if min_score is not None:
                        cat_data = cat_data[cat_data['score'] >= min_score]
                    if max_score is not None:
                        cat_data = cat_data[cat_data['score'] <= max_score]
                
                available_count = len(cat_data)
                
                if available_count == 0:
                    print(f"⚠️  No images match filters, skipping...")
                    continue
                
                print(f"📊 {available_count} images available")
                
                # Show the smart grid
                smart_grid_robust(
                    category=category, 
                    n_images=n_images_per_category, 
                    sort_by=sort_by,
                    min_score=min_score,
                    max_score=max_score
                )
                
                successful_categories += 1
                
                # Add some spacing between categories (except for the last one)
                if i < len(categories) - 1:
                    print("\n" + "="*60)
                    
            except Exception as e:
                print(f"❌ Error processing category {category}: {e}")
                continue
        
        print(f"\n✅ Successfully processed {successful_categories}/{len(categories)} categories")
        
    except Exception as e:
        print(f"❌ Critical error in explore_all_categories_robust: {e}")

In [ ]:
def display_images_with_descriptions(n_images=5, 
                                   category=None, 
                                   min_score=None, 
                                   max_score=None,
                                   sort_by='score_high',
                                   figsize_per_image=(8, 10)):
    """Display images with descriptions below them (vertical layout)."""
    if analyzer is None:
        print("❌ No results loaded")
        return
    
    if 'summary' not in analyzer.df.columns:
        print("❌ No summary data available in results")
        return
    
    try:
        # Filter data
        df_filtered = analyzer.df.copy()
        
        if category:
            df_filtered = df_filtered[df_filtered['category'] == category]
        
        if 'score' in df_filtered.columns:
            if min_score is not None:
                df_filtered = df_filtered[df_filtered['score'] >= min_score]
            if max_score is not None:
                df_filtered = df_filtered[df_filtered['score'] <= max_score]
        
        # Filter for images that have summaries
        df_filtered = df_filtered[df_filtered['summary'].notna()]
        
        if len(df_filtered) == 0:
            print("❌ No images with summaries found matching the filters")
            return
        
        # Sort and limit
        if sort_by == 'score_high' and 'score' in df_filtered.columns:
            df_filtered = df_filtered.sort_values('score', ascending=False)
        elif sort_by == 'score_low' and 'score' in df_filtered.columns:
            df_filtered = df_filtered.sort_values('score', ascending=True)
        elif sort_by == 'random':
            df_filtered = df_filtered.sample(frac=1)
        
        df_subset = df_filtered.head(n_images)
        
        print(f"📖 Displaying {len(df_subset)} images with descriptions")
        if category:
            print(f"🏷️  Category: {category}")
        if min_score is not None or max_score is not None:
            score_range = f"Score range: {min_score or 'any'} - {max_score or 'any'}"
            print(f"📊 {score_range}")
        print("=" * 80)
        
        for idx, (_, row) in enumerate(df_subset.iterrows()):
            try:
                print(f"\n🖼️  Image {idx + 1}/{len(df_subset)}: {row['filename']}")
                
                # Display score and category if available
                info_parts = []
                if 'score' in row and pd.notna(row['score']) and row['score'] >= -2:
                    info_parts.append(f"Score: {row['score']:.1f}")
                if 'category' in row and pd.notna(row['category']):
                    info_parts.append(f"Category: {row['category']}")
                
                if info_parts:
                    print(f"ℹ️  {' | '.join(info_parts)}")
                
                # Create subplot with image on top, text below
                fig, (ax_img, ax_text) = plt.subplots(2, 1, figsize=figsize_per_image, 
                                                     gridspec_kw={'height_ratios': [2, 1]})
                
                # Display image
                try:
                    img_path = '../' + row['image_path']
                    if Path(img_path).exists():
                        img = Image.open(img_path)
                        ax_img.imshow(img)
                        ax_img.axis('off')
                        
                        # Create title with metadata
                        title_parts = [f"{row['filename']}"]
                        if info_parts:
                            title_parts.append(' | '.join(info_parts))
                        ax_img.set_title('\n'.join(title_parts), fontsize=12, pad=15)
                    else:
                        ax_img.text(0.5, 0.5, f"Image not found:\n{row['filename']}", 
                                   ha='center', va='center', transform=ax_img.transAxes)
                        ax_img.axis('off')
                except Exception as e:
                    ax_img.text(0.5, 0.5, f"Error loading image:\n{str(e)[:50]}", 
                               ha='center', va='center', transform=ax_img.transAxes)
                    ax_img.axis('off')
                
                # Display summary text below
                summary_text = str(row['summary']) if pd.notna(row['summary']) else "No summary available"
                
                # Add reasoning if available
                if 'reasoning' in row and pd.notna(row['reasoning']):
                    reasoning_text = str(row['reasoning'])
                    # Truncate very long reasoning text
                    # if len(reasoning_text) > 300:
                    #     reasoning_text = reasoning_text[:300] + "..."
                    summary_text += f"\n\n🧠 Reasoning: {reasoning_text}"
                
                # Display text with better formatting
                ax_text.text(0.02, 0.98, summary_text, transform=ax_text.transAxes, 
                           fontsize=10, ha='left', va='top', wrap=True,
                           bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.3))
                ax_text.axis('off')
                
                plt.tight_layout()
                plt.show()
                
                # Add separator
                print("-" * 80)
                
            except Exception as e:
                print(f"❌ Error displaying image {idx + 1}: {e}")
                continue
        
        print(f"✅ Completed displaying {len(df_subset)} images with descriptions")
        
    except Exception as e:
        print(f"❌ Error in display_images_with_descriptions: {e}")

In [ ]:
def display_images_with_descriptions_compact(n_images=5, 
                                           category=None, 
                                           min_score=None, 
                                           max_score=None,
                                           sort_by='score_high',
                                           figsize_per_image=(6, 8)):
    """Compact version with smaller images and concise descriptions."""
    if analyzer is None:
        print("❌ No results loaded")
        return
    
    if 'summary' not in analyzer.df.columns:
        print("❌ No summary data available in results")
        return
    
    try:
        # Filter data (same as above)
        df_filtered = analyzer.df.copy()
        
        if category:
            df_filtered = df_filtered[df_filtered['category'] == category]
        
        if 'score' in df_filtered.columns:
            if min_score is not None:
                df_filtered = df_filtered[df_filtered['score'] >= min_score]
            if max_score is not None:
                df_filtered = df_filtered[df_filtered['score'] <= max_score]
        
        df_filtered = df_filtered[df_filtered['summary'].notna()]
        
        if len(df_filtered) == 0:
            print("❌ No images with summaries found matching the filters")
            return
        
        # Sort and limit
        if sort_by == 'score_high' and 'score' in df_filtered.columns:
            df_filtered = df_filtered.sort_values('score', ascending=False)
        elif sort_by == 'score_low' and 'score' in df_filtered.columns:
            df_filtered = df_filtered.sort_values('score', ascending=True)
        elif sort_by == 'random':
            df_filtered = df_filtered.sample(frac=1)
        
        df_subset = df_filtered.head(n_images)
        
        print(f"📖 Displaying {len(df_subset)} images with descriptions (compact)")
        if category:
            print(f"🏷️  Category: {category}")
        print("=" * 60)
        
        for idx, (_, row) in enumerate(df_subset.iterrows()):
            try:
                # Create compact layout
                fig, (ax_img, ax_text) = plt.subplots(2, 1, figsize=figsize_per_image, 
                                                     gridspec_kw={'height_ratios': [3, 2]})
                
                # Display image
                try:
                    img_path = '../' + row['image_path']
                    if Path(img_path).exists():
                        img = Image.open(img_path)
                        ax_img.imshow(img)
                        ax_img.axis('off')
                        
                        # Compact title
                        title_parts = []
                        if 'score' in row and pd.notna(row['score']) and row['score'] >= -2:
                            title_parts.append(f"Score: {row['score']:.1f}")
                        if 'category' in row and pd.notna(row['category']):
                            title_parts.append(f"Cat: {row['category']}")
                        
                        title = f"{row['filename']}"
                        if title_parts:
                            title += f" | {' | '.join(title_parts)}"
                        
                        ax_img.set_title(title, fontsize=10, pad=10)
                    else:
                        ax_img.text(0.5, 0.5, f"Image not found", 
                                   ha='center', va='center', transform=ax_img.transAxes)
                        ax_img.axis('off')
                except Exception as e:
                    ax_img.text(0.5, 0.5, f"Error loading image", 
                               ha='center', va='center', transform=ax_img.transAxes)
                    ax_img.axis('off')
                
                # Display concise summary
                summary_text = str(row['summary']) if pd.notna(row['summary']) else "No summary available"
                
                # Truncate long summaries for compact view
                if len(summary_text) > 200:
                    summary_text = summary_text[:200] + "..."
                
                ax_text.text(0.05, 0.95, summary_text, transform=ax_text.transAxes, 
                           fontsize=9, ha='left', va='top', wrap=True,
                           bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.2))
                ax_text.axis('off')
                
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"❌ Error displaying image {idx + 1}: {e}")
                continue
        
        print(f"✅ Completed compact display of {len(df_subset)} images")
        
    except Exception as e:
        print(f"❌ Error in display_images_with_descriptions_compact: {e}")


print("📖 Updated description viewers loaded!")
print("\n🚀 Available functions:")
print("  display_images_with_descriptions()         # Image on top, description below")
print("  display_images_with_descriptions_compact() # Smaller, more concise version")
print("\n📋 Quick examples:")
print("  display_images_with_descriptions(3)                    # Top 3 with descriptions")
print("  display_images_with_descriptions_compact(5, min_score=7.0)  # Compact high-quality")

In [ ]:
print("🛡️ Robust functions loaded with enhanced error handling!")
print("📖 New description viewer available!")
print("\n🚀 Updated functions:")
print("  smart_grid_robust()                    # Enhanced grid with error handling")
print("  explore_all_categories_robust()        # Robust category exploration")

In [ ]:
smart_grid_robust(category='A', n_images=170, grid_cols=6, sort_by='random', 
                     min_score=None, max_score=2)

In [ ]:
# 🛡️ Test the robust category exploration
explore_all_categories_robust(
    n_images_per_category=16,
    sort_by='score_high',
    min_score=None
)

In [ ]:
# 🛡️ Test the robust category exploration
explore_all_categories_robust(
    n_images_per_category=16,
    sort_by='score_low',
    min_score=None
)

In [ ]:
# 📖 NEW: Display images with their descriptions
# Change these variables to explore different combinations

N_DESCRIPTIONS = 15           # Number of images to show with descriptions
DESCRIPTION_CATEGORY = None  # Filter by category (None, 'A', 'B', etc.)
DESCRIPTION_MIN_SCORE = None  # Minimum score (None to disable)
DESCRIPTION_MAX_SCORE = None # Maximum score (None to disable)
DESCRIPTION_SORT = 'score_high'  # 'score_high', 'score_low', 'random'

display_images_with_descriptions(
    n_images=N_DESCRIPTIONS,
    category=DESCRIPTION_CATEGORY,
    min_score=DESCRIPTION_MIN_SCORE,
    max_score=DESCRIPTION_MAX_SCORE,
    sort_by=DESCRIPTION_SORT
)